In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [56]:
data = pd.read_excel(r"579004_579091.xlsx").dropna(axis=1)
data

,Stock1,Stock2
0,2.60,1.91
1,8.38,4.64
2,-0.06,-1.47
3,4.03,0.12
4,-5.23,-4.48
...,...,...
1995,-6.92,0.82
1996,1.85,2.36
1997,-8.98,-6.95
1998,-6.01,-6.29


In [61]:
# 2) Historical Simulation
VAR_PERCENTAGE = 99

var_99_stock1 = np.percentile(data["Stock1"], 100 - VAR_PERCENTAGE)  
var_99_stock2 = np.percentile(data["Stock2"], 100 - VAR_PERCENTAGE)  

print(f"VaR at {VAR_PERCENTAGE}% for Stock 1 is: {var_99_stock1}")
print(f"VaR at {VAR_PERCENTAGE}% for Stock 2 is: {var_99_stock2}")

# Different approach
data_sorted = data.copy()
data_sorted.iloc[:] = np.sort(data.mul(-1), axis=0)

idx = int(np.ceil(len(data_sorted) * VAR_PERCENTAGE / 100)) - 1
var_stocks = data_sorted.iloc[idx, :]

print(var_stocks) # They agree

VaR at 99% for Stock 1 is: -14.0903
VaR at 99% for Stock 2 is: -11.8401
Stock1    14.09
Stock2    11.84
Name: 1979, dtype: float64


In [96]:
# 4) EVT Approach
k = 100
n = len(data)

data_sorted = data.copy()
data_sorted.iloc[:] = np.sort(data.mul(-1), axis=0)

log_sum = 0
for i in range(1, k + 1):
    log_sum += np.log(data_sorted.iloc[(n - i), :])

tail_index_estimate = ((1 / k) * log_sum - np.log(data_sorted.iloc[(n - k - 1), :]))**-1 # Hill estimator
print(f"Alpha hat = {tail_index_estimate}")

var_evt = data_sorted.iloc[(n - k - 1), :] * ((k / (n * (1 - VAR_PERCENTAGE/100)))**(1 / tail_index_estimate))
print(f"VaRs at {VAR_PERCENTAGE}% using the EVT approach with k = {k}: {var_evt}")


Alpha hat = Stock1    3.457562
Stock2    3.604361
dtype: float64
VaRs at 99% using the EVT approach with k = 100: Stock1    13.315674
Stock2    11.440223
dtype: float64
